# Exploration
---

### Content
- Overview
- Prep-work
- Data Loading
- Data Cleansing & Quality Review
- Data Enrichments
- Findings & ideas
- Quick Summary & Next Steps

### Overview
This notebook serves as a first approach to the data set, performing various investigation, and data quality check.

With a cleaned resulting dataframe, we will compute high level metrics based on the business objectives on a sale

> Examine the mutual fund flows across brokerages, geographies, fund categories and
territories to provide actionable insights that will help Firm X increase its flows and elevate firm X’s
market share

### Prep-work
After saving the data in the `RawData` folder, navigate into the folder within `powershell`, and run the following command:

`Get-ChildItem *zip | Expand-Archive -Destination .`

Your `RawData` directory should now contain all the data we need in `.txt` format instead of the incoming `.zip` files.

---
### Data Loading

The following block of code will read in all the data and combine it together into one dataframe

Resulting dataframe should contain the following:

| Column Name || Descriptions |
| ----------- || ----------- |
| Broker Name [broker_name] || The intermediary firm where the fund was sold |
| City & State [city_state] || Location of the intermediary firm’s city and state |
| Territory [territory] || Each territory is the responsibility of a single sales person—Prefix “I” denotes the “IBD” channel and “W” denotes the “Wirehouse” channel |
| Fund Category [fund_category] || The Morning Star category of the fund that was sold |
| Firm X Sales [firm_x_sales] || Amount of funds sold at the specific intermediary by Firm X |
| Total Industry Sales [total_industry_sales] || Total amount of funds sold (including those by Firm X) at that intermediary |
| Email Open [email_open] || Whether or not an email from Firm X has been opened by a broker |
| Web Visit [web_visit] || Whether or not Firm X’s website has been visited by a broker | 
| Webcast Attendee [webcast_attendee] || Whether or not a webcast by Firm X has been attended by a broker |
| Marketing Engaged [marketing_engaged] || Whether or not any of Firm X’s marketing material have been engaged with by a broker |



In [1]:
# Importing libraries
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import numpy as np

import glob
from datetime import datetime 

In [2]:
# Scanning for all the data files
dataPaths = glob.glob("RawData\\*.txt")

# Loading in the data
listOfFrames = []
for i in dataPaths:
    tdf = pd.read_csv(i, sep=";")
    tdf['source_file'] = i  # Adding source file as a column for ease of tracking
    listOfFrames.append(tdf)
    
# Combining all the dataframes
df = pd.concat(listOfFrames, ignore_index=True)

In [3]:
# Checking the top of the dataframe
df.head(5)

,broker_name,city_state,territory,fund_category,firm_x_sales,total_industry_sales,email_open,web_visit,webcast_attendee,marketing_engaged,source_file
0,Broker0001,"OVERLAND PARK, KS",I47,Intermediate-Term Bond,0.0,179227.86,N,Y,N,Engaged,RawData\Analytics_Data1.txt
1,Broker0002,"GREAT NECK, NY",I48,Short-Term Bond,0.0,21956.00,Y,Y,Y,Engaged,RawData\Analytics_Data1.txt
2,Broker0003,"PITTSBURGH, PA",I09,Diversified Emerging Mkts,0.0,53223.03,N,N,N,Not engaged,RawData\Analytics_Data1.txt
3,Broker0004,"FAIRFIELD, CT",I25,Foreign Large Blend,0.0,595799.49,Y,N,N,Engaged,RawData\Analytics_Data1.txt
4,Broker0005,"LAWTON, OK",W27,Bank Loan,0.0,0.00,Y,N,N,Engaged,RawData\Analytics_Data1.txt


In [4]:
# Checking the bottom of the dataframe
df.tail(5)

,broker_name,city_state,territory,fund_category,firm_x_sales,total_industry_sales,email_open,web_visit,webcast_attendee,marketing_engaged,source_file
1734105,Broker0047,"GLENELG, MD",I42,World Stock,0.0,0.00,N,N,N,Not engaged,RawData\Analytics_Data2.txt
1734106,Broker0019,"WEXFORD, PA",I09,Multialternative,0.0,30558.28,N,N,N,Not engaged,RawData\Analytics_Data2.txt
1734107,Broker0089,"VIRGINIA BEACH, VA",I13,Inflation-Protected Bond,0.0,0.00,Y,Y,Y,Engaged,RawData\Analytics_Data2.txt
1734108,Broker0050,"LAS CRUCES, NM",I04,Foreign Large Growth,0.0,0.00,N,N,N,Engaged,RawData\Analytics_Data2.txt
1734109,Broker0022,"ROCHESTER HILLS, MI",I08,Bank Loan,0.0,5000.00,N,N,N,Not engaged,RawData\Analytics_Data2.txt


In [5]:
# Checking a random sample of the dataframe
df.sample(5)

,broker_name,city_state,territory,fund_category,firm_x_sales,total_industry_sales,email_open,web_visit,webcast_attendee,marketing_engaged,source_file
473522,Broker0005,"CHICAGO, IL",W10,Short Government,0.00,779993.50,Y,Y,Y,Engaged,RawData\Analytics_Data1.txt
536779,Broker0001,"MOREHEAD CITY, NC",I26,Equity Precious Metals,0.00,400.00,N,N,N,Not engaged,RawData\Analytics_Data1.txt
1643758,Broker0016,"BLOOMFIELD HILLS, MI",I08,Short-Term Bond,0.00,5471620.73,Y,Y,Y,Engaged,RawData\Analytics_Data2.txt
1550299,Broker0004,"WALL TOWNSHIP, NJ",I49,Foreign Large Blend,0.00,530629.49,Y,N,N,Engaged,RawData\Analytics_Data2.txt
778203,Broker0001,"PORTLAND, CT",I25,Large Growth,6723.89,116934.59,N,N,N,Not engaged,RawData\Analytics_Data1.txt


In [6]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1734110 entries, 0 to 1734109
Data columns (total 11 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   broker_name           1734110 non-null  object 
 1   city_state            1734110 non-null  object 
 2   territory             1734110 non-null  object 
 3   fund_category         1734110 non-null  object 
 4   firm_x_sales          1734110 non-null  float64
 5   total_industry_sales  1734110 non-null  float64
 6   email_open            1734110 non-null  object 
 7   web_visit             1734110 non-null  object 
 8   webcast_attendee      1734110 non-null  object 
 9   marketing_engaged     1734110 non-null  object 
 10  source_file           1734110 non-null  object 
dtypes: float64(2), object(9)
memory usage: 145.5+ MB


---
### Data Cleansing & Quality Review

#### High level clean up
Dropping the duplicates, cleaning up the white spaces

In [7]:
df = df.drop_duplicates()

print("-----{After dropping the duplicates}-----\n")
df.info(verbose=True, show_counts=True)

-----{After dropping the duplicates}-----

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1721310 entries, 0 to 1734109
Data columns (total 11 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   broker_name           1721310 non-null  object 
 1   city_state            1721310 non-null  object 
 2   territory             1721310 non-null  object 
 3   fund_category         1721310 non-null  object 
 4   firm_x_sales          1721310 non-null  float64
 5   total_industry_sales  1721310 non-null  float64
 6   email_open            1721310 non-null  object 
 7   web_visit             1721310 non-null  object 
 8   webcast_attendee      1721310 non-null  object 
 9   marketing_engaged     1721310 non-null  object 
 10  source_file           1721310 non-null  object 
dtypes: float64(2), object(9)
memory usage: 157.6+ MB


In [8]:
# Run a for-loop to go through string columns
# and strip the leading and trailing whitespaces

for i in df.columns:
    if is_string_dtype(df[i]):   # if it's a string column
        df[i] = df[i].str.strip()   # strip out the white spaces
    else:
        pass
    
# resetting index after the drop duplicates
df = df.reset_index(drop=True)

#### [broker_name]
Looks like majority of this column starts with a string broker, then follow with a 4-digits number.

In [9]:
# Checking if any of this field doesn't start with "Broker"
df[df['broker_name'].str[:6] != "Broker"]


,broker_name,city_state,territory,fund_category,firm_x_sales,total_industry_sales,email_open,web_visit,webcast_attendee,marketing_engaged,source_file


In [10]:
# Checking how many columns it slited outm, and patterns
df['broker_name'].str.split("Broker", expand=True).head(3)

,0,1
0,,0001
1,,0002
2,,0003


In [11]:
# Check the second part of the field if they're numbers
is_numeric_dtype(df['broker_name'].str.split("Broker", expand=True)[1].astype(float))

True

In [12]:
# Appears that broker is not unique
# Assuming each row is a lead, Broker0001 looks like a significant player
# It may worth calculating the conversion rate by Brokers
df['broker_name'].value_counts()

Broker0001    430039
Broker0016    139999
Broker0022     99783
Broker0009     67648
Broker0003     62568
               ...  
Broker1856         1
Broker1460         1
Broker1864         1
Broker1783         1
Broker0857         1
Name: broker_name, Length: 1885, dtype: int64

#### [city_state]
This column contains a mix of strings, and is in high risk for data quality fails. First, we will split up the states as a new column, and check both individually.

In [13]:
# checking what is splitted out by a comma, looks like majority of them
# has a city, follow by a state
# it appears that there're rows containing more than one comma
df['city_state'].str.split(", ", expand=True).head(5)

,0,1,2
0,OVERLAND PARK,KS,None
1,GREAT NECK,NY,None
2,PITTSBURGH,PA,None
3,FAIRFIELD,CT,None
4,LAWTON,OK,None


In [14]:
# Getting the [2] from split for investigation
df['city_state'].str.split(", ", expand=True)[2].value_counts()

DC    17
TN    14
Name: 2, dtype: int64

In [15]:
# Going to check all three of 
mask = df['city_state'].str.split(", ", expand=True)[2].isnull() == False

df.loc[mask, "city_state"].unique()

array(['KNOXVILLE, TN, TN', 'WASHINGTON, DC, DC'], dtype=object)

In [16]:
# Cleaning up the duplicated states
df['city_state'] = df['city_state'].str.replace("WASHINGTON, DC, DC", "WASHINGTON DC, DC")
df['city_state'] = df['city_state'].str.replace("KNOXVILLE, TN, TN", "KNOXVILLE, TN")

In [17]:
# Validating the comma splits into two columns
SplitCounts = len(df['city_state'].str.split(", ", expand=True).columns)
print("[city_state] column-split check: ", SplitCounts == 2)

[city_state] column-split check:  True


In [18]:
# Cleaning the KNOXVILLE, TN into one entity
mask = df['city_state'].str.contains("KNOXVILLE")

df.loc[mask, 'city_state'].unique()

array(['KNOXVILLE, TN', 'KNOXVILLE, IA'], dtype=object)

In [19]:
# Checking if the city is a zip code, or is actually a city name, 
mask = df['city_state'].str.split(", ", expand=True)[0].str.isnumeric()

df.loc[mask, "city_state"].unique()

array(['64150, MO', '95678, CA', '15801, PA'], dtype=object)

In [20]:
# Searching for the corresponding cities manually for this project
# Will replace mechanism with any firm approved geo API as reference data

cleaningDict = {
    '64150, MO' : 'RIVERSIDE, MO',
    '95678, CA' : 'ROSEVILLE, CA',
    '15801, PA' : 'DUBOIS, PA'
}

# Performing the cleaning
df['city_state'] = df['city_state'].replace(cleaningDict)

In [21]:
# Validating if the cleaning process works.
mask = df['city_state'].str.split(", ", expand=True)[0].str.isnumeric()

# This should return an empty array
df.loc[mask, "city_state"].unique()

array([], dtype=object)

#### [territory]

Validating if the ID starts with either I or W as a letter, then follow with a numerical ID

In [22]:
# Checking if thet prefix is a character per describe
mask = df['territory'].str[0].str.isalpha() == False
mask1 = df['territory'].str[0] != "I"
mask2 = df['territory'].str[0] != "W"

print("Non-alphabet:", len(df.loc[mask, 'territory'].unique()))
print("Non-I/W:", len(df.loc[mask1&mask2, 'territory'].unique()))

Non-alphabet: 0
Non-I/W: 0


In [23]:
# Checking if the following is an actual number
mask = df['territory'].str[1:].str.isnumeric() == False

print("Non-numeric id:", len(df.loc[mask, 'territory'].unique()))

Non-numeric id: 0


#### [fund_category]

Quick check if there is non-alphabetical names in fund categories. The reason why we are using `.str.isnumeric()` instead of `is_numeric_dtype` because numerical values can be stored as string, but string cannot be stored as `float` or `integer`.

In [24]:
# Checking if any fund category names are non alphabets
mask = df['fund_category'].str.isnumeric() == True

print("Abnormal fund category counts:", len(df.loc[mask, 'fund_category'].unique()))

Abnormal fund category counts: 0


#### [firm_x_sales], [total_industry_sales]

These columns are shown as `float64`, so we will only use simple checks on data type consistencies. 

In [25]:
print("Is Firm X Sales Column Numeric:", is_numeric_dtype(df['firm_x_sales']))
print("Is Total Industry Sales Column Numeric:", is_numeric_dtype(df['total_industry_sales']))

Is Firm X Sales Column Numeric: True
Is Total Industry Sales Column Numeric: True


In [26]:
# Since individual sale can coincidentally having the same amount, 
# we will assume there's more false negatives
# Inspecting the Total Industry Sales amount below
print("Count of unique Brokers:", len(df['broker_name'].unique()))
print("Count of unique industry sale amounts:", len(df['total_industry_sales'].unique()))
print("Total line items in data:", len(df))

Count of unique Brokers: 1885
Count of unique industry sale amounts: 953406
Total line items in data: 1721310


As we can see, we have to further investigate the mechanism of how the total amount of funds sold by intermediaries since the unique number of brokres do not match up.

First, we will do aggregations to see if the market share % make sense with a groupby on intermediary level

In [27]:
# Aggregate them by intermediaries for 
investigate_df = df.groupby('broker_name').agg({
                                'firm_x_sales' : 'sum',
                                'total_industry_sales' : 'sum'
                            })

# See if firm_x_sales's max is not exceeding total_industry_sales
investigate_df['pct'] = investigate_df['firm_x_sales'] / investigate_df['total_industry_sales']

In [28]:
# dropping NA to remove the 0/0 scenarios
broker_group = investigate_df.dropna().sort_values('pct', ascending=False) # The sorting will show the max pct on top
broker_group

,firm_x_sales,total_industry_sales,pct
broker_name,,,
Broker1055,77529.00,77529.00,1.0
Broker1734,42000.00,42000.00,1.0
Broker1365,20150.00,20150.00,1.0
Broker1486,50000.00,50000.00,1.0
Broker0473,431054.32,431054.32,1.0
...,...,...,...
Broker0748,0.00,7794263.37,0.0
Broker0747,0.00,2816000.00,0.0
Broker0746,0.00,1150900.00,0.0


In [29]:
# Getting initial overview on the market share
print("Overall market share distribution: \n")
print(broker_group['pct'].describe())
print('-' * 20)


split_point = broker_group['pct'].describe()['std']
print("One standard deviation away from min: \n")
print(broker_group[broker_group['pct'] > split_point]['pct'].describe())

Overall market share distribution: 

count    1545.000000
mean        0.024518
std         0.107327
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000009
max         1.000000
Name: pct, dtype: float64
--------------------
One standard deviation away from min: 

count    72.000000
mean      0.389893
std       0.319300
min       0.107620
25%       0.138313
50%       0.240639
75%       0.538186
max       1.000000
Name: pct, dtype: float64


Since the data is highly skewed, and it's a continuous variable, we will shift the observation window by one standard deviation and get a clearer view on the distribution. 

In business sense, we are neglecting the bottom 10% at this section; this particular group can be out top prospect after further investigations.

#### [email_open], [web_visit], [webcast_attendee], [marketing_engaged]

All of these columns seem to be categorical variables, we will do simple validation with `value_counts`.

In [30]:
# Checking if the columns are actually binary by doing value counts
# looks like the binary data can be modified into 1s and 0s for efficiencies
binary_columns = ['email_open', 'web_visit', 'webcast_attendee', 'marketing_engaged']

for i in binary_columns:
    print(f"<< {i} >>\n")
    print(df[i].value_counts())
    print('-' * 20)

<< email_open >>

N    930339
Y    790971
Name: email_open, dtype: int64
--------------------
<< web_visit >>

N    1085541
Y     635769
Name: web_visit, dtype: int64
--------------------
<< webcast_attendee >>

N    1407189
Y     314121
Name: webcast_attendee, dtype: int64
--------------------
<< marketing_engaged >>

Engaged        991683
Not engaged    729627
Name: marketing_engaged, dtype: int64
--------------------


---
### Data Enrichments


#### High Level Overview of all columns

In [31]:
df.describe(include='all').loc[['unique','top','freq'],:]

,broker_name,city_state,territory,fund_category,firm_x_sales,total_industry_sales,email_open,web_visit,webcast_attendee,marketing_engaged,source_file
unique,1885,8359,70,95,NaN,NaN,2,2,2,2,2
top,Broker0001,"NEW YORK, NY",I29,Multi Asset Alloc,NaN,NaN,N,N,N,Engaged,RawData\Analytics_Data2.txt
freq,430039,11194,46246,57602,NaN,NaN,930339,1085541,1407189,991683,868228


#### Modifying data type within columns

First, we will convert the binary columns into 1s and 0s for ease of analysis, computational efficiencies.

In [32]:
# Intiating the mapping for conversion
binary_encoding_map = {
    'Y' : 1,
    'N' : 0,
    'Engaged' : 1,
    'Not engaged' : 0
}

In [33]:
# Performing the enhancement
df[binary_columns] = df[binary_columns].replace(binary_encoding_map)
df.head(5)

,broker_name,city_state,territory,fund_category,firm_x_sales,total_industry_sales,email_open,web_visit,webcast_attendee,marketing_engaged,source_file
0,Broker0001,"OVERLAND PARK, KS",I47,Intermediate-Term Bond,0.0,179227.86,0,1,0,1,RawData\Analytics_Data1.txt
1,Broker0002,"GREAT NECK, NY",I48,Short-Term Bond,0.0,21956.00,1,1,1,1,RawData\Analytics_Data1.txt
2,Broker0003,"PITTSBURGH, PA",I09,Diversified Emerging Mkts,0.0,53223.03,0,0,0,0,RawData\Analytics_Data1.txt
3,Broker0004,"FAIRFIELD, CT",I25,Foreign Large Blend,0.0,595799.49,1,0,0,1,RawData\Analytics_Data1.txt
4,Broker0005,"LAWTON, OK",W27,Bank Loan,0.0,0.00,1,0,0,1,RawData\Analytics_Data1.txt


#### Splitting up the locations

Since city names duplicates in the US, we will keep the `city_state` column to carry the meaning of city, and only going to add the `state` column from the split.

In [34]:
df['state'] = df['city_state'].str.split(", ", expand=True)[1]
df.head(5)

,broker_name,city_state,territory,fund_category,firm_x_sales,total_industry_sales,email_open,web_visit,webcast_attendee,marketing_engaged,source_file,state
0,Broker0001,"OVERLAND PARK, KS",I47,Intermediate-Term Bond,0.0,179227.86,0,1,0,1,RawData\Analytics_Data1.txt,KS
1,Broker0002,"GREAT NECK, NY",I48,Short-Term Bond,0.0,21956.00,1,1,1,1,RawData\Analytics_Data1.txt,NY
2,Broker0003,"PITTSBURGH, PA",I09,Diversified Emerging Mkts,0.0,53223.03,0,0,0,0,RawData\Analytics_Data1.txt,PA
3,Broker0004,"FAIRFIELD, CT",I25,Foreign Large Blend,0.0,595799.49,1,0,0,1,RawData\Analytics_Data1.txt,CT
4,Broker0005,"LAWTON, OK",W27,Bank Loan,0.0,0.00,1,0,0,1,RawData\Analytics_Data1.txt,OK


In [35]:
# Validating if all states are in 2-letters fashion
df['state'].str.len().unique()

array([2], dtype=int64)

#### Splitting up the channels

Similar as the location, we will extract the channel as an extra feature to observe potential patterns.

First, we will confirm if the channels are actually going binary.

In [36]:
# Due to the clean pattern, we will just utilize the simple substring to extract the channel
df['territory'].str[0].unique()

array(['I', 'W'], dtype=object)

In [37]:
df['i_or_w'] =df['territory'].str[0]

Since it is in fact binary in nature, we will encode the feature in 1s and 0s for ease of usages with the following configurations.

    I : 1
    W : 0


In [38]:
I_OR_W = {
    "I" : 1,
    "W" : 0
}

df['i_or_w'] = df['i_or_w'].replace(I_OR_W)

In [39]:
df.head(5)

,broker_name,city_state,territory,fund_category,firm_x_sales,total_industry_sales,email_open,web_visit,webcast_attendee,marketing_engaged,source_file,state,i_or_w
0,Broker0001,"OVERLAND PARK, KS",I47,Intermediate-Term Bond,0.0,179227.86,0,1,0,1,RawData\Analytics_Data1.txt,KS,1
1,Broker0002,"GREAT NECK, NY",I48,Short-Term Bond,0.0,21956.00,1,1,1,1,RawData\Analytics_Data1.txt,NY,1
2,Broker0003,"PITTSBURGH, PA",I09,Diversified Emerging Mkts,0.0,53223.03,0,0,0,0,RawData\Analytics_Data1.txt,PA,1
3,Broker0004,"FAIRFIELD, CT",I25,Foreign Large Blend,0.0,595799.49,1,0,0,1,RawData\Analytics_Data1.txt,CT,1
4,Broker0005,"LAWTON, OK",W27,Bank Loan,0.0,0.00,1,0,0,1,RawData\Analytics_Data1.txt,OK,0


In [40]:
print("Unique value of each fields: \n")
print(df.describe(include='all').loc['unique',:])

Unique value of each fields: 

broker_name             1885
city_state              8359
territory                 70
fund_category             95
firm_x_sales             NaN
total_industry_sales     NaN
email_open               NaN
web_visit                NaN
webcast_attendee         NaN
marketing_engaged        NaN
source_file                2
state                     53
i_or_w                   NaN
Name: unique, dtype: object


There is `53` unique states getting splitted up, and the US only has `51` states (including Washington, DC), so we will do one extra step to check what the remaining 2 are.

In [41]:
# Creating a list of US states
# Due to time constrain, the following list is copied from an open source github
# so we can reduce unnecessary data entries

us_states = [("AL","Alabama"),("AK","Alaska"),("AZ","Arizona"),("AR","Arkansas"),("CA", "California"),("CO", "Colorado"),
("CT","Connecticut"),("DC","Washington DC"),("DE","Delaware"),("FL","Florida"),("GA","Georgia"),
("HI","Hawaii"),("ID","Idaho"),("IL","Illinois"),("IN","Indiana"),("IA","Iowa"),("KS","Kansas"),("KY","Kentucky"),
("LA","Louisiana"),("ME","Maine"),("MD","Maryland"),("MA","Massachusetts"),("MI","Michigan"),("MN","Minnesota"),
("MS","Mississippi"),("MO","Missouri"),("MT","Montana"),("NE","Nebraska"),("NV","Nevada"),("NH","New Hampshire"),
("NJ","New Jersey"),("NM","New Mexico"),("NY","New York"),("NC","North Carolina"),("ND","North Dakota"),("OH","Ohio"),
("OK","Oklahoma"),("OR","Oregon"),("PA","Pennsylvania"),("RI","Rhode Island"),("SC","South Carolina"),("SD","South Dakota"),
("TN","Tennessee"),("TX","Texas"),("UT","Utah"),("VT","Vermont"),("VA","Virginia"),("WA","Washington"),("WV","West Virginia"),
("WI","Wisconsin"),("WY","Wyoming")]

us_state_list = [i[0] for i in us_states]

Overlaying the difference with a left join

In [42]:
# Using set to grab unique values of the two, and finding the differences between
set(df['state']) - set(us_state_list)

{'GU', 'PR'}

`GU` and `PR` stands for `Guam` and `Puerto Rico` correspondingly, which are US territories, and can be included in our data.

#### Efficient Sale

Sales lead data is highly imbalanced in nature, in order to align the analytics with our business goal, we will have to decicide how to measure a success.

Based on the `80-20 rule`, for which `80%` fo the profit are generated by `20%` of the effort, we will draw a line at the `twenty-percentile` as a label.

Since market share will increase as sales increase, and total sales by broker (including other funds) is out of our control, we will focus on `effective sale` instead of a market share study.

*Note: this is different from the market share observation, the prior **cut by standard deviation on market share does NOT account for our business efforts**.*

In [43]:
# Initiating a filter to exclude all the no sales generated record
mask = df['firm_x_sales'] > 0

# Utilizing the filter to only look at the successful sales
# and find the twenty percentile as a cutting line
# to only include the top 80% of sales
significant_cut = df[mask]['firm_x_sales'].quantile(0.2)

print(f"We will label anything above {significant_cut} as an 'Effective Sale'.")

We will label anything above 2161.968 as an 'Effective Sale'.


In [44]:
# Intiating a filter with the twenty percentile cut
mask = df['firm_x_sales'] > significant_cut

# Performing the filter and label
df['effective_sale'] = 0
df.loc[mask,'effective_sale'] = 1

df['effective_sale'].unique()

array([0, 1], dtype=int64)

In [45]:
# Initiate a filter to show ineffective sale that we can neglect.
mask = df['firm_x_sales'] > 0
mask1 = df['firm_x_sales'] < significant_cut

# Validating if the cut works
# Below should return 0
df.loc[mask & mask1, 'effective_sale'].unique()

array([0], dtype=int64)

#### Driver Pattern

Since a lead generation is likely not a single trigger, most of the time, a conversion is driven by multiple factors.

Therefore, a `driver pattern` maybe beneficial in the study. Based KISS (Keep it simple and stupid), we will just create a new column containing the information instead of generating combination maps.

In [46]:
# Creating a simple column filter for cleaniness
driver_columns = [
                'email_open', 'web_visit',
                'webcast_attendee', 'marketing_engaged',
            ]

# The result of the driver_pattern should be a column of tuples
# The choice of tuples was because we don't want any flexibilities in it
df['driver_pattern'] = df[driver_columns].apply(tuple,axis=1)

In [47]:
# Validating if the driver pattern reflects the actual columns 
mask1 = df['email_open'] == 0
mask2 = df['web_visit'] == 0
mask3 = df['webcast_attendee'] == 0
mask4 = df['marketing_engaged'] == 0

df[mask1 & mask2 & mask3 & mask4].equals(df[df['driver_pattern'] == (0,0,0,0)])

True

---
### Quick findings & ideas

#### Demonstrating the dataframe with the enrichments

In [48]:
# Using a random sample to show a more holistic view
df.sample(5)

,broker_name,city_state,territory,fund_category,firm_x_sales,total_industry_sales,email_open,web_visit,webcast_attendee,marketing_engaged,source_file,state,i_or_w,effective_sale,driver_pattern
1082716,Broker0023,"SCOTTSDALE, AZ",I04,Large Growth,0.00,900.00,0,0,0,1,RawData\Analytics_Data2.txt,AZ,1,0,"(0, 0, 0, 1)"
291209,Broker0001,"CLINTON, IL",I07,Multi Asset Alloc,1079.68,2543741.53,1,1,1,1,RawData\Analytics_Data1.txt,IL,1,0,"(1, 1, 1, 1)"
1125907,Broker0049,"PLAINVIEW, NY",I48,Large Growth,0.00,0.00,0,0,0,0,RawData\Analytics_Data2.txt,NY,1,0,"(0, 0, 0, 0)"
1706663,Broker0261,"NORFOLK, MA",I11,Nontrad Multisect Bond,0.00,11400.00,1,1,0,1,RawData\Analytics_Data2.txt,MA,1,0,"(1, 1, 0, 1)"
834219,Broker0016,"ROCHESTER, NY",I10,Small Blend,0.00,4427.15,1,1,0,1,RawData\Analytics_Data1.txt,NY,1,0,"(1, 1, 0, 1)"


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1721310 entries, 0 to 1721309
Data columns (total 15 columns):
 #   Column                Dtype  
---  ------                -----  
 0   broker_name           object 
 1   city_state            object 
 2   territory             object 
 3   fund_category         object 
 4   firm_x_sales          float64
 5   total_industry_sales  float64
 6   email_open            int64  
 7   web_visit             int64  
 8   webcast_attendee      int64  
 9   marketing_engaged     int64  
 10  source_file           object 
 11  state                 object 
 12  i_or_w                int64  
 13  effective_sale        int64  
 14  driver_pattern        object 
dtypes: float64(2), int64(6), object(7)
memory usage: 197.0+ MB


#### Feature setups for studies

Creating a library of frequently used combination for cleaniness.

In [50]:
# Creating a column libary for ease of navigating
column_libraries = {
    'driver_pattern' : ['driver_pattern'],
    'driver_columns' : [
                'email_open', 'web_visit',
                'webcast_attendee', 'marketing_engaged',
            ],
    'dimension_columns' : [
                'broker_name', 'city_state', 'territory', 
                'fund_category', 'state', 'i_or_w'
            ],
    'measurement_columns' : ['firm_x_sales','total_industry_sales', 'effective_sale']
}

Validating if the library is created correctly

In [51]:
# Checker to validate if we have covered all the columns except the source_file
num_col = 0
for key, value in column_libraries.items():
    num_col += len(value) # for each column category, count number of values


check = len(df.columns) - 1 == num_col
print(f"{check} - dataframe has {len(df.columns) - 1}, and our library has {num_col}")

True - dataframe has 14, and our library has 14


#### Driver studies

The following for-loop will iterate all the driver columns to calculate potential conversion rate(since we do not know if the driver features happenning sequences)

In [52]:
all_sales = df['firm_x_sales'].sum()
market_share = all_sales / df['total_industry_sales'].sum() 
all_leads = len(df)

effective_sale_count = len(df[df['effective_sale']==1])
effective_rate = effective_sale_count / all_leads

print(f"Total Sales: {'${:,.2f}'.format(all_sales)}")
print(f"Current Market Share: {market_share}")
print(f"Total Leads: {'{:,}'.format(all_leads)}")
print(f"Total Effective Sale: {'{:,}'.format(effective_sale_count)}")
print(f"Effective Sale Rate: {effective_rate}")
# print(f"Effective Sale Rate: {}")

Total Sales: $19,735,928,897.53
Current Market Share: 0.0421795951858633
Total Leads: 1,721,310
Total Effective Sale: 153,432
Effective Sale Rate: 0.08913676211722467


In [53]:
for i in column_libraries.get("driver_columns"):
    # creating a subset of the dataframe that has the the feature is on.
    feature_mask = (df[i] == 1)
    data_extract = df[feature_mask].copy()
    sale_mask = data_extract['effective_sale'] == 1
    
    #calculating the conversion rates
    total_activate = len(data_extract)
    total_convert = len(data_extract[sale_mask])
    total_fail = len(data_extract[sale_mask==False])
    conversion_rate = total_convert / total_activate
    fail_rate = total_fail / total_activate
    
    #Calculating total sales by this one feature (including in effectives)
    related_sales = data_extract[data_extract['firm_x_sales'] > 0]['firm_x_sales'].sum()
    
                      
    print(f"[{i}] feature report:\n")
    print(f"Total {i}: {'{:,}'.format(total_activate)}")
    print(f"Successful Conversion: {'{:,}'.format(total_convert)}")
    print(f"Fail Conversion: {'{:,}'.format(total_fail)}\n")
    print(f"[{i}] related sales: {'${:,.2f}'.format(related_sales)}\n" )
    print(f"Conversion Rate: {conversion_rate} | Fail Rate: {fail_rate}")
    print('-' * 20)

[email_open] feature report:

Total email_open: 790,971
Successful Conversion: 86,310
Fail Conversion: 704,661

[email_open] related sales: $14,982,506,094.96

Conversion Rate: 0.10911904481959515 | Fail Rate: 0.8908809551804049
--------------------
[web_visit] feature report:

Total web_visit: 635,769
Successful Conversion: 74,284
Fail Conversion: 561,485

[web_visit] related sales: $13,886,446,322.94

Conversion Rate: 0.11684117973666536 | Fail Rate: 0.8831588202633346
--------------------
[webcast_attendee] feature report:

Total webcast_attendee: 314,121
Successful Conversion: 40,071
Fail Conversion: 274,050

[webcast_attendee] related sales: $9,578,735,211.85

Conversion Rate: 0.1275654922784532 | Fail Rate: 0.8724345077215467
--------------------
[marketing_engaged] feature report:

Total marketing_engaged: 991,683
Successful Conversion: 99,067
Fail Conversion: 892,616

[marketing_engaged] related sales: $16,067,663,358.15

Conversion Rate: 0.09989785042195944 | Fail Rate: 0.9001

Most of the drivers are showing `approximately 10% conversion rate`, and the related sale amount is insignificant to differentiate which is more important.

Due to the nature of multi-reasoning for a sale, we believe the above metrics are only a high level overview, and we need to study combinations of drivers.

In [54]:
# Generating a pivot view by driver pattern
df.groupby('driver_pattern').agg({
                    'firm_x_sales':'sum',
                    'effective_sale':'sum', # use sum here due to it's 1s and 0s
                }).sort_values('firm_x_sales', ascending=False)

,firm_x_sales,effective_sale
driver_pattern,,
"(1, 1, 1, 1)",9.160197e+09,35363
"(1, 1, 0, 1)",4.318073e+09,33994
"(0, 0, 0, 0)",3.668266e+09,54365
"(1, 0, 0, 1)",1.167220e+09,13140
"(0, 0, 0, 1)",6.069117e+08,7123
"(0, 1, 0, 1)",3.967237e+08,4739
"(1, 0, 1, 1)",3.370167e+08,3813
"(0, 0, 1, 1)",7.006868e+07,707
"(0, 1, 1, 1)",1.145322e+07,188


Interestingly, ranked number 3 does not have any driver signals. This will require extra business side understanding to disect the insights.

The above driver patterns are based on a linear view, where we are only looking at all the signals simultaneously without any conditioning orders. **We will go into the sequential insights with Machine Learning technique in later session.**

#### Geographic studies

Giving a high level overview of the split based on geographic locations, since `city_state` are over specific, we will take a step back with the `state` as target.

In [55]:
# Creating a pivot view to summaize sales and leads by states
geo_df = df.groupby('state').agg({
                    'firm_x_sales' : 'sum',
                    'total_industry_sales' : 'sum',
                    'effective_sale' : 'sum',
                    'broker_name' : 'nunique',
                    'source_file' : 'count' # this is just taking in a dummie column for count since we assumed each row is a lead
                })

geo_df = geo_df.rename(columns={
                    'source_file':'num_leads',
                    'broker_name' : 'num_brokers'
                })

In [56]:
# Calculating the marekt share
geo_df['market_share'] = geo_df['firm_x_sales'] / geo_df['total_industry_sales']

# Calculating conversions
geo_df['conversion_rate'] = geo_df['effective_sale'] / geo_df['num_leads']

# effective_sale to market_share ratio
geo_df['C/M ratio'] = geo_df['conversion_rate'] / geo_df['market_share']

# Calculating conversion rate to number of lead ratios
geo_df['L/C ratio'] = geo_df['num_leads'] / (geo_df['conversion_rate'] * 100)

# Calculating conversion rate to number of broker ratios
geo_df['B/C ratio'] = geo_df['num_brokers'] / geo_df['conversion_rate']

First, we will look at high level to see which state we have the highest market shares, highest sale and effective sale.

In [57]:
geo_df.sort_values("market_share", ascending=False).head(5)

,firm_x_sales,total_industry_sales,effective_sale,num_brokers,num_leads,market_share,conversion_rate,C/M ratio,L/C ratio,B/C ratio
state,,,,,,,,,,
WY,5.925773e+07,7.657000e+08,441,37,4244,0.077390,0.103911,1.342693,408.424853,356.072562
ID,1.468532e+08,1.902948e+09,1227,82,10481,0.077171,0.117069,1.516999,895.284116,700.441728
ND,9.196558e+07,1.295104e+09,752,47,6841,0.071010,0.109925,1.548023,622.330864,427.562500
NE,2.810385e+08,4.025522e+09,1884,93,16493,0.069814,0.114230,1.636204,1443.837840,814.144904
MT,1.311891e+08,1.895790e+09,910,63,8342,0.069200,0.109087,1.576391,764.713890,577.523077


In [58]:
geo_df.sort_values("firm_x_sales", ascending=False).head(5)

,firm_x_sales,total_industry_sales,effective_sale,num_brokers,num_leads,market_share,conversion_rate,C/M ratio,L/C ratio,B/C ratio
state,,,,,,,,,,
CA,2.071481e+09,5.682379e+10,13118,464,167067,0.036454,0.078519,2.153903,21277.163050,5909.367891
TX,1.453554e+09,3.287236e+10,10643,276,109131,0.044218,0.097525,2.205544,11190.054647,2830.043785
NY,1.344243e+09,3.424746e+10,6924,320,98900,0.039251,0.070010,1.783656,14126.530907,4570.768342
FL,1.033135e+09,2.442436e+10,7657,307,90620,0.042299,0.084496,1.997564,10724.806582,3633.321144
PA,8.745406e+08,2.030313e+10,6262,227,86889,0.043074,0.072069,1.673136,12056.369085,3149.760939


In [59]:
geo_df.sort_values("effective_sale", ascending=False).head(5)

,firm_x_sales,total_industry_sales,effective_sale,num_brokers,num_leads,market_share,conversion_rate,C/M ratio,L/C ratio,B/C ratio
state,,,,,,,,,,
CA,2.071481e+09,5.682379e+10,13118,464,167067,0.036454,0.078519,2.153903,21277.163050,5909.367891
TX,1.453554e+09,3.287236e+10,10643,276,109131,0.044218,0.097525,2.205544,11190.054647,2830.043785
FL,1.033135e+09,2.442436e+10,7657,307,90620,0.042299,0.084496,1.997564,10724.806582,3633.321144
IL,8.160872e+08,2.433820e+10,7465,262,78149,0.033531,0.095523,2.848775,8181.200537,2742.804823
NY,1.344243e+09,3.424746e+10,6924,320,98900,0.039251,0.070010,1.783656,14126.530907,4570.768342


As we can see, the general sum of sale and volumn of effective sales are closely resembling each other. We can guess NY has more small investment amounts that contribute to the overall sales.

The true decision of target is between PA and IL, **where one has more effective sale (less labor cost), one generates more revenue.**

Therefore, some combined sort may help seeing the picture better.

In [60]:
geo_df.sort_values("C/M ratio", ascending=False).head(10)

,firm_x_sales,total_industry_sales,effective_sale,num_brokers,num_leads,market_share,conversion_rate,C/M ratio,L/C ratio,B/C ratio
state,,,,,,,,,,
PR,1.586548e+06,7.835043e+07,35,12,533,0.020249,0.065666,3.242867,81.168286,182.742857
HI,4.816480e+07,1.610102e+09,580,65,6696,0.029914,0.086619,2.895583,773.041655,750.413793
IL,8.160872e+08,2.433820e+10,7465,262,78149,0.033531,0.095523,2.848775,8181.200537,2742.804823
AL,1.365460e+08,3.997292e+09,1614,89,17147,0.034160,0.094127,2.755512,1821.682831,945.528501
GA,3.913784e+08,1.183115e+10,3303,192,38565,0.033080,0.085648,2.589079,4502.752725,2241.743869
WI,4.319057e+08,1.246226e+10,4586,153,53256,0.034657,0.086112,2.484697,6184.477837,1776.748365
KS,2.550201e+08,6.235962e+09,2222,118,22265,0.040895,0.099798,2.440340,2231.009113,1182.389739
AR,9.395788e+07,2.261476e+09,1191,85,11763,0.041547,0.101250,2.436983,1161.781436,839.508816
WA,5.223675e+08,1.149104e+10,4682,147,43845,0.045459,0.106785,2.349061,4105.903513,1376.594404


The above table is sorted by the `C/M ratio` in decending order, where it shows states with high convesions but low market share, and **poses an opportunity**. `IL` stands out as a potentially good target.

>*General recommendation: Continue current strategy, increase expense in the top 10 list since this is a higher guarantee area.*

After that, we want to troubleshoot locations that has **high number of leads, but low conversion rates** to troubleshoot it. If there's low number of leads, it becomes a separate issue. When there is a lead, but we fail to convert, there is something going wrong.

In [61]:
geo_df.sort_values("conversion_rate", ascending=False).head(5)

,firm_x_sales,total_industry_sales,effective_sale,num_brokers,num_leads,market_share,conversion_rate,C/M ratio,L/C ratio,B/C ratio
state,,,,,,,,,,
SC,3.144022e+08,5.109964e+09,2867,122,23686,0.061527,0.121042,1.967289,1956.841981,1007.914894
ID,1.468532e+08,1.902948e+09,1227,82,10481,0.077171,0.117069,1.516999,895.284116,700.441728
NE,2.810385e+08,4.025522e+09,1884,93,16493,0.069814,0.114230,1.636204,1443.837840,814.144904
MO,4.944677e+08,1.030903e+10,4791,156,43059,0.047965,0.111266,2.319754,3869.917514,1402.046337
AK,5.358333e+07,9.503047e+08,349,32,3140,0.056385,0.111146,1.971192,282.510029,287.908309


Solely looking at the `conversion_rate` is not very useful since it does not account of number of leads.

In [62]:
geo_df.sort_values("L/C ratio", ascending=False).head(10)

,firm_x_sales,total_industry_sales,effective_sale,num_brokers,num_leads,market_share,conversion_rate,C/M ratio,L/C ratio,B/C ratio
state,,,,,,,,,,
GU,0.000000e+00,0.000000e+00,0,1,1,NaN,0.000000,NaN,inf,inf
CA,2.071481e+09,5.682379e+10,13118,464,167067,0.036454,0.078519,2.153903,2.127716e+04,5909.367891
NY,1.344243e+09,3.424746e+10,6924,320,98900,0.039251,0.070010,1.783656,1.412653e+04,4570.768342
PA,8.745406e+08,2.030313e+10,6262,227,86889,0.043074,0.072069,1.673136,1.205637e+04,3149.760939
TX,1.453554e+09,3.287236e+10,10643,276,109131,0.044218,0.097525,2.205544,1.119005e+04,2830.043785
FL,1.033135e+09,2.442436e+10,7657,307,90620,0.042299,0.084496,1.997564,1.072481e+04,3633.321144
OH,8.327550e+08,1.906714e+10,6504,244,76202,0.043675,0.085352,1.954261,8.927959e+03,2858.746617
IL,8.160872e+08,2.433820e+10,7465,262,78149,0.033531,0.095523,2.848775,8.181201e+03,2742.804823
MI,7.899716e+08,1.668428e+10,6149,210,65577,0.047348,0.093768,1.980382,6.993565e+03,2239.578793


Since `GU` does not have significant number of leads, we will neglect the particular observation. The `L/C ratio` shows the high number of leads but relatively poor conversions.

> *General Recommendation: perform content review on drivers (e.g. email contents, webcast...etc.) we want quality leads because any steps further poses a cost*

Similar approach towards a driver, a higher number of brokers, the higher the chance we can make a conversion. 

In [63]:
geo_df.sort_values("B/C ratio", ascending=False).head(10)

,firm_x_sales,total_industry_sales,effective_sale,num_brokers,num_leads,market_share,conversion_rate,C/M ratio,L/C ratio,B/C ratio
state,,,,,,,,,,
GU,0.000000e+00,0.000000e+00,0,1,1,NaN,0.000000,NaN,inf,inf
CA,2.071481e+09,5.682379e+10,13118,464,167067,0.036454,0.078519,2.153903,2.127716e+04,5909.367891
NY,1.344243e+09,3.424746e+10,6924,320,98900,0.039251,0.070010,1.783656,1.412653e+04,4570.768342
FL,1.033135e+09,2.442436e+10,7657,307,90620,0.042299,0.084496,1.997564,1.072481e+04,3633.321144
PA,8.745406e+08,2.030313e+10,6262,227,86889,0.043074,0.072069,1.673136,1.205637e+04,3149.760939
OH,8.327550e+08,1.906714e+10,6504,244,76202,0.043675,0.085352,1.954261,8.927959e+03,2858.746617
TX,1.453554e+09,3.287236e+10,10643,276,109131,0.044218,0.097525,2.205544,1.119005e+04,2830.043785
MA,4.203565e+08,1.349091e+10,2822,198,40321,0.031158,0.069988,2.246204,5.761102e+03,2829.042523
NJ,5.514320e+08,1.441080e+10,3554,202,48747,0.038265,0.072907,1.905310,6.686185e+03,2770.651097


#### Fund Category studies

This is a more complicated area to observe, since risk tolerance and financial literacy varies between different people, and fund performance comes into the picture objectively. The dollar amount can simply be a portion of the portfolio.

Since investment decision is based on the financial education, the following session will have more focus on the `channel` and possibly the `webcast`

In [64]:
# Creating a pivot view to summaize sales and leads by Channel
i_or_w_df = df.groupby('i_or_w').agg({
                    'firm_x_sales' : 'sum',
                    'total_industry_sales' : 'sum',
                    'effective_sale' : 'sum',
                    'broker_name' : 'nunique',
                    'source_file' : 'count' # this is just taking in a dummie column for count since we assumed each row is a lead
                })

i_or_w_df = i_or_w_df.rename(columns={
                    'source_file':'num_leads',
                    'broker_name' : 'num_brokers'
                })

In [65]:
# Calculating the marekt share
i_or_w_df['market_share'] = i_or_w_df['firm_x_sales'] / i_or_w_df['total_industry_sales']

# Calculating conversions
i_or_w_df['conversion_rate'] = i_or_w_df['effective_sale'] / i_or_w_df['num_leads']

# effective_sale to market_share ratio
i_or_w_df['C/M ratio'] = i_or_w_df['conversion_rate'] / i_or_w_df['market_share']

# Calculating conversion rate to number of broker ratios
i_or_w_df['B/C ratio'] = i_or_w_df['num_brokers'] / i_or_w_df['conversion_rate']

In [66]:
i_or_w_df.sort_values("firm_x_sales", ascending=False)

,firm_x_sales,total_industry_sales,effective_sale,num_brokers,num_leads,market_share,conversion_rate,C/M ratio,B/C ratio
i_or_w,,,,,,,,,
1,1.453638e+10,2.934510e+11,140446,1879,1602218,0.049536,0.087657,1.769568,21435.766216
0,5.199552e+09,1.744512e+11,12986,6,119092,0.029805,0.109042,3.658482,55.024796


There is significantly higher number of brokers in the IBD channel than the Wirehouse channel. 

In [67]:
# Getting the IBD datafram e
ibd_df = df[df['i_or_w'] == 1].copy()
wirehouse_df = df[df['i_or_w'] == 0].copy()

In [68]:
wire_product_df = wirehouse_df.groupby('fund_category').agg({
                            'firm_x_sales' : 'sum',
                            'effective_sale' : 'sum',
                            'webcast_attendee' : 'sum',
                            'source_file' : 'count' # this is just taking in a dummie column for count since we assumed each row is a lead
                        })

wire_product_df = wire_product_df.rename(columns={
                    'source_file':'num_leads',
                })

In [69]:
# Calculating conversions
wire_product_df['conversion_rate'] = wire_product_df['effective_sale'] / wire_product_df['num_leads']


In [70]:
wire_product_df.sort_values("conversion_rate", ascending=False).head(5)

,firm_x_sales,effective_sale,webcast_attendee,num_leads,conversion_rate
fund_category,,,,,
Large Value,8.452901e+08,1527,1154,2096,0.728531
Multi Asset Alloc,8.744617e+08,1298,1154,2111,0.614874
Intermediate-Term Bond,4.989560e+08,1114,1150,2052,0.542885
Mid-Cap Blend,4.047635e+08,1031,1133,1949,0.528989
Large Growth,2.437621e+08,1060,1153,2117,0.500709


In [71]:
# Calculating the best selling funds in the IBD channel
wire_product_df.sort_values("firm_x_sales", ascending=False).head(5)

,firm_x_sales,effective_sale,webcast_attendee,num_leads,conversion_rate
fund_category,,,,,
Nontrad Multisect Bond,1.047918e+09,923,1150,2060,0.448058
Multi Asset Alloc,8.744617e+08,1298,1154,2111,0.614874
Large Value,8.452901e+08,1527,1154,2096,0.728531
Intermediate-Term Bond,4.989560e+08,1114,1150,2052,0.542885
Mid-Cap Blend,4.047635e+08,1031,1133,1949,0.528989


As we compare the `conversion_rate` sort and the `firm_x_sales` sort, we can see the prior contains more equity portfolios, it is probably driven by the Sharpe ratios and higher portfolio returns. And the higher bond product inquiries may indicate that the market demand for bond is higher than equity.

We will check on an approximate of bond products

In [72]:
# creating a subset of data to compare bond vs not bond
mask = wire_product_df.index.str.contains("Bond")
wire_product_df['is_bond'] = 0
wire_product_df.loc[mask, 'is_bond'] = 1

In [73]:
wire_product_df = wire_product_df.groupby('is_bond').agg({
                        'num_leads' : 'sum',
                        'effective_sale' : 'sum',
                        'firm_x_sales' : 'sum'
                    })

In [74]:
wire_pct_bond = wire_product_df.loc[1,'effective_sale'] / wire_product_df['effective_sale'].sum()

print(f"Percent of effective sale from bonds: {wire_pct_bond}")

Percent of effective sale from bonds: 0.21615586015709226


We will have to compare % of bond sale with the IBD channel

In [75]:
ibd_product_df = ibd_df.groupby('fund_category').agg({
                            'firm_x_sales' : 'sum',
                            'effective_sale' : 'sum',
                            'webcast_attendee' : 'sum',
                            'source_file' : 'count' # this is just taking in a dummie column for count since we assumed each row is a lead
                        })

ibd_product_df = ibd_product_df.rename(columns={
                    'source_file':'num_leads',
                })

In [76]:
# Calculating conversions
ibd_product_df['conversion_rate'] = ibd_product_df['effective_sale'] / ibd_product_df['num_leads']

# Calculating conversion rate to number of lead ratios
ibd_product_df['L/C ratio'] = ibd_product_df['num_leads'] / (ibd_product_df['conversion_rate'] * 100)

In [77]:
ibd_product_df.sort_values("conversion_rate", ascending=False).head(5)

,firm_x_sales,effective_sale,webcast_attendee,num_leads,conversion_rate,L/C ratio
fund_category,,,,,,
Intermediate-Term Bond,2.782350e+09,15419,5921,48185,0.319996,1505.800782
Mid-Cap Blend,1.077344e+09,10826,5047,34376,0.314929,1091.547548
Large Value,2.469860e+09,15344,6084,52743,0.290920,1812.971878
Emerging-Markets Local-Currency Bond,6.115933e+07,1638,1382,5883,0.278429,211.292363
Small Blend,8.220397e+08,7642,5201,35959,0.212520,1692.030465


In [78]:
ibd_product_df.sort_values("firm_x_sales", ascending=False).head(5)

,firm_x_sales,effective_sale,webcast_attendee,num_leads,conversion_rate,L/C ratio
fund_category,,,,,,
Intermediate-Term Bond,2.782350e+09,15419,5921,48185,0.319996,1505.800782
Large Value,2.469860e+09,15344,6084,52743,0.290920,1812.971878
Multi Asset Alloc,1.193315e+09,8420,6173,55491,0.151736,3657.067792
Large Growth,1.175802e+09,11332,6136,54798,0.206796,2649.859516
Mid-Cap Blend,1.077344e+09,10826,5047,34376,0.314929,1091.547548


In [79]:
# creating a subset of data to compare bond vs not bond
mask = ibd_product_df.index.str.contains("Bond")
ibd_product_df['is_bond'] = 0
ibd_product_df.loc[mask, 'is_bond'] = 1

In [80]:
ibd_product_df = ibd_product_df.groupby('is_bond').agg({
                        'num_leads' : 'sum',
                        'effective_sale' : 'sum',
                        'firm_x_sales' : 'sum'
                    })



In [81]:
ibd_pct_bond = ibd_product_df.loc[1,'effective_sale'] / ibd_product_df['effective_sale'].sum()

print(f"Percent of effective sale from bonds: {ibd_pct_bond}")

Percent of effective sale from bonds: 0.28267803995841817


Bond fund sale from IBD is about `28%`, approximately `7%` more than the wirehouse at `26%`. Assuming the data is current, it means the financial education within IBD (individual brokers) are not as up to date as our wirehouse.

> *General Recommendation: Targeting the IBD, we will deploy more bond contents*

---
### Quick Summary & Next Steps

1. Sales/Leads drivers are approcimately indifferent in conversion
2. We will move on to the **next session to use machine learning to further investigate** the inter-relationship between them
3. From the geographical breakdown, we will target high C/M ratio areas to scale up by continuing the current strategies. (e.g. IL, AL, GA)
4. The high B/C ratio areas briefly overlaps the high L/C, we will have to review the sales process to identify reason for low conversions. (e.g. CA, NY, PA, TX)
5. On high level, IBD is marginally selling more bonds vs equities compare to Wirehouse. Moving forward, we will produce more targeted content.

#### Next...
1. Refactoring the cleaning procedure code into Python module for ease of teamwork, and scalability.
2. Utilizing machine learning technique to reveal sales driver lifecycle.